In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!wget --quiet https://raw.githubusercontent.com/pytorch/vision/release/0.12/references/detection/engine.py
!wget --quiet https://raw.githubusercontent.com/pytorch/vision/release/0.12/references/detection/utils.py
!wget --quiet https://raw.githubusercontent.com/pytorch/vision/release/0.12/references/detection/transforms.py
!wget --quiet https://raw.githubusercontent.com/pytorch/vision/release/0.12/references/detection/coco_eval.py
!wget --quiet https://raw.githubusercontent.com/pytorch/vision/release/0.12/references/detection/coco_utils.py

In [ ]:
# from torch_snippets import *
from IPython import display
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchinfo import summary
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T, models, datasets
from xml.etree import ElementTree as et
import torchvision
import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.patches as patches
import tqdm
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
print(torchvision.__version__)

In [ ]:
df = pd.read_csv('/kaggle/input/lgg-mri-segmentation/kaggle_3m/data.csv')#.head(4)
print(f'Количество уникальных пациентов: {df.Patient.nunique()}')
print('-----------------------------------------------------------')
df.head(4)

# **1. Создаем списки путей картинок и масок**

In [ ]:
brain_scans = []
mask_files = glob.glob('../input/lgg-mri-segmentation/kaggle_3m/*/*_mask*')

for i in mask_files:
    brain_scans.append(i.replace('_mask',''))

print(brain_scans[:2])
print(mask_files[:2])
print(len(brain_scans))
print(len(mask_files))

-------------------------------------------------
-------------------------------------------------
-------------------------------------------------

# **2. Передварительный анализ**

In [ ]:
#Lets plot some samples
rows,cols=3,3
fig=plt.figure(figsize=(10,10))
for i in range(1,rows*cols+1):
    fig.add_subplot(rows,cols,i)
    img_path=brain_scans[i]
    msk_path=mask_files[i]
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    msk=cv2.imread(msk_path)
    plt.imshow(img)
    plt.imshow(msk,alpha=0.3)
    plt.xticks([]), plt.yticks([])
plt.show()

# **3. Смотрим содержимое каналов масок**

In [ ]:
rows,cols=1,3
fig=plt.figure(figsize=(10,10))
for i in np.arange(1,4):
    fig.add_subplot(rows,cols,i)
    msk_path=mask_files[2]
    msk=cv2.imread(msk_path)
    plt.imshow(msk.transpose(-1,0,1)[i-1],alpha=0.3)
    plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# читаем картинку маски, 4меняем каналы местасми на (3,250,250),
an = cv2.imread(mask_files[2], 1).transpose(2,0,1)#/255
r,g,b = an

# координаты пикселей маски
nzs = np.nonzero(r==255)#==255)
instances = np.unique(g[nzs])
nzs

**Уникальные значения в каждом канале:**

In [ ]:
lst = []
for _ in [0,1,2]:
    k = r,g,b
    h = 'rgb'
    for i in k[_]:
        for j in i:
            if j not in lst: lst.append(j)
    print(f'Значения в канале {h[_]}: {lst}')

# **4. Создаем список annots с массивами array аннотаций:**

In [ ]:
# если на картинке нет людей, т.е. цифр 255 в слое 'r', то картинка пропускается, иначе добавляется
from tqdm import tqdm
annots = []
for ann in tqdm(mask_files):
    _ann = cv2.imread(ann, 1).transpose(2,0,1)#/255
    r,g,b = _ann
    if 255 not in np.unique(r): continue # отсеиваем маски в которых нет аномалии
    annots.append(ann)
print(f'Количество элементов: {len(annots)}')
print(f'Количество уникальных элементов: {len(set(annots))}')
print(f'Пример содержимого: {annots[:1]}')

# **5. train_test_split**

In [ ]:
from sklearn.model_selection import train_test_split
_annots = [i.split('/')[-1].replace('.tif','') for i in annots] # список масок без расширений: ['ADE_train_00016892', 'ADE_train_00004335', ....
# _annots = stems(annots)
trn_items, val_items = train_test_split(_annots, test_size=0.2, random_state=2)
print(f'Длина trn_items: {len(trn_items)}')
print(f'Длина val_items: {len(val_items)}')
print(f'Пример содержимого trn_items: {trn_items[:1]}')
print(f'Пример содержимого val_items: {val_items[:1]}')

# **6. Аугментация**

In [ ]:
def get_transform(train):
    image_transforms = []
    image_transforms.append(T.PILToTensor())
#     if train:
#         image_transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(image_transforms)

# **7. Создаем класс для создания пакетов**

In [ ]:
# items - список названий файлов-масок без расширения
# way_pic - список путей до файлов-изображений с расширениями
# way_ann - список путей до файлов-масок с расширениями

class MasksDataset(Dataset):
    def __init__(self, items, transforms, way_pic, way_ann):#, N): 
        self.items = items # список картинок: ['ADE_train_00016892', 'ADE_train_00004335', ....
        self.transforms = transforms
        self.way_pic = way_pic
        self.way_ann = way_ann
#         self.N = len(items) # ограничивает количество картинок для рассмотрения
    def get_mask(self, path):
        an = cv2.imread(path, 1).transpose(-1,0,1)/255 # делаем из картинки array и меняем каналы метстами, стало: (3,450,640)
        r,g,b = an # вытаскиваем отдельные каналы
        nzs = np.nonzero(r==1.0) # вытаскивает координатытточек людей, т.е. значений 4: (array([319, 319, 319, ..., 500, 500, 500]), array([139, 140, 141, ..., 429, 430, 613]))
        instances = np.unique(g[nzs]) # в слое r находим коорд людей, в слое g для каждого человека свои цифры, короче здесь список цифр, которые обозначают людей
        masks = np.zeros((len(instances), *r.shape), float) # (39, 512, 683),т.е. на картинке 39 людей, здесь созданы 39 нулевых масок
        for ix,_id in enumerate(instances):
            masks[ix] = g==_id # маска состоит из 0 и 1, 0 - это черный фон, 1 - это человечек
        return masks # список масок на которых по одному человечку, обозначенному единичкой
    def __getitem__(self, ix):
        _id = self.items[ix] # название картинки , здесь 'TCGA_DU_7010_19860307_43_mask'
        for i in self.way_pic:
            if _id[:-5] in i:
#                 print(i)
                img_path = i
        for i in self.way_ann:
            if _id[:-5] in i:
#                 print(i)
                mask_path = i
        masks = self.get_mask(mask_path) # это список масок людей на этой картинке, каждаякартинка
        
        obj_ids = np.arange(1, len(masks)+1) # array список по кол-ву картинок array([ 1,  2,  3,  4,  5,..., 36, 37, 38, 39])
        img = Image.open(img_path).convert("RGB")
        num_objs = len(obj_ids) # количество масок (39)
        boxes = []
        for i in range(num_objs):
            obj_pixels = np.where(masks[i]) # (array([472, 472, 472, 472,..., 478, 478, 478]), array([343, 344, 345, 346,..., 345, 346, 347, 348]))
                                            # т.к. человечки это единички, то это True, здесь np.where возвращает координаты всех пикселей True (человечков)
            xmin = np.min(obj_pixels[1])
            xmax = np.max(obj_pixels[1])
            ymin = np.min(obj_pixels[0])
            ymax = np.max(obj_pixels[0])
            if (((xmax-xmin)<=10) | (ymax-ymin)<=10): # я так понимаю,что этот цикл для избежания ошибки =0
                xmax = xmin+10
                ymax = ymin+10
            boxes.append([xmin, ymin, xmax, ymax])
        boxes = torch.as_tensor(boxes, dtype=torch.float32) # список коробок переводим в тензор
        labels = torch.ones((num_objs,), dtype=torch.int64) # создаем список из единичек длиной по числу людей на картинке tensor([1, 1, 1, 1, ..... 1, 1, 1, 1])
        masks = torch.as_tensor(masks, dtype=torch.uint8) # список масок переводим в тензор
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]) #площадь: tensor([100., 100., 154., 143., 407., 456., 396., 256.,  90., 119., 180., 162.,
        # 200., 115., 243., 225., 196., 455., 147., 208., 216., 410.,  95., 115.,90., 147., 100., 132., 160.,  52., 216., 225., 312.,  80., 100., 100., 100., 100., 100.])
        # это площади всех коробок людей, здесь 39 людей и выходит столько же коробок
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64) # создаем список из ноликов длиной по числу людей на картинке tensor([0, 0, 0, 0, ..... 0, 0, 0, 0])
        image_id = torch.tensor([ix]) # tensor([0]) - индекс-тензор картинки
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
#         print(target)
        if self.transforms is not None:
            img = self.transforms(img)
        if (img.dtype == torch.float32) or (img.dtype == torch.uint8) :
            img = img/255.
        return img, target
    def __len__(self):
        return len(self.items)
    def choose(self):
        return self[randint(len(self))]
    def collate_fn(self, batch):
        return tuple(zip(*batch))

In [ ]:
dataset = MasksDataset(trn_items, get_transform(train=False),brain_scans,mask_files)#, N=len(trn_items))

In [ ]:
for i in dataset:
    print(i[0])
    break

In [ ]:
dataset = MasksDataset(trn_items, get_transform(train=False), brain_scans, mask_files)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=dataset.collate_fn)

In [ ]:
for i in data_loader:
    print(i[1])
    break

# **7. Функции левые для обучения**

In [ ]:
%%capture
!pip install coco-eval
!pip install coco-utils

In [ ]:
import math
import sys
import time

import torch
import torchvision.models.detection.mask_rcnn
import utils
from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset


def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq, scaler=None):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger


def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types


@torch.inference_mode()
def evaluate(model, data_loader, device):
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"

    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)

    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"]: output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator

In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq, scaler=None):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger

# **8. Модель, оптимайзер, lr и др.**

In [ ]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # получить количество входных признаков для классификатора
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,num_classes)
    return model

In [ ]:
model = get_model_instance_segmentation(2).to(device)
model

In [ ]:
num_classes = 2
model = get_model_instance_segmentation(num_classes).to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

In [ ]:
num_epochs = 4

trn_history = []
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    res = train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    trn_history.append(res)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    res = evaluate(model, data_loader_test, device=device)

# **дальше еще не рабочее**

In [ ]:
!pip install utils
import utils

In [ ]:
# from torch_snippets import *
# from torch_snippets.inspector import inspect
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

# from engine import train_one_epoch, evaluate
import utils
# import transforms as T

In [ ]:
from detection import utils

In [ ]:
dataset = MasksDataset(trn_items, get_transform(train=False), brain_scans, mask_files)
dataset_test = MasksDataset(val_items, get_transform(train=False), brain_scans, mask_files)

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=0,
    collate_fn=dataset.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=dataset_test.collate_fn)

In [ ]:
for i in data_loader:
    print(i)
    break

In [ ]:
st = '/kaggle/input/lgg-mri-segmentation/lgg-mri-segmentation/kaggle_3m/TCGA_DU_7306_19930512/TCGA_DU_7306_19930512_35_mask.tif'
st.split('/')[-1].replace('_mask','')[:-4]

In [ ]:
'/kaggle/input/lgg-mri-segmentation/lgg-mri-segmentation/kaggle_3m/TCGA_DU_7306_19930512/TCGA_DU_7306_19930512_35_mask.tif'